Movie Studio

In the last 10 years, which genre was the most profitable at the box office. profits between genres.

Most popular english movies filtered by user rating??? filters - year, english, gross profit, user rating, best director 3 recommendations? comedy vs drama

In [230]:
from scipy import stats

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import pandas as pd
import numpy as np

import sqlite3
import zipfile

import warnings
warnings.filterwarnings('ignore')

In [231]:
movie_budget = pd.read_csv("data/tn.movie_budgets.csv.gz", sep= ",")

In [232]:
movie_budget.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [233]:
movie_budget.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [234]:
movie_budget_copy = movie_budget.copy()

movie_budget_copy['release_date'] = pd.to_datetime(movie_budget_copy['release_date'], errors='coerce')

movie_budget_copy['release_year'] = pd.to_datetime(movie_budget_copy['release_date']).dt.year

# Extract the month from the 'release_date' column
movie_budget_copy['release_month'] = movie_budget_copy['release_date'].dt.month

# Display the DataFrame with the new 'release_month' column
movie_budget_copy


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month
0,1,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,776,345,279",2009,12
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011,5
2,3,2019-06-07,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",2019,6
3,4,2015-05-01,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",2015,5
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",2017,12
...,...,...,...,...,...,...,...,...
5777,78,2018-12-31,Red 11,"$7,000",$0,$0,2018,12
5778,79,1999-04-02,Following,"$6,000","$48,482","$240,495",1999,4
5779,80,2005-07-13,Return to the Land of Wonders,"$5,000","$1,338","$1,338",2005,7
5780,81,2015-09-29,A Plague So Pleasant,"$1,400",$0,$0,2015,9


In [235]:
conn = sqlite3.connect('im.db.zip')
#specify the path to the zip file
zip_file_path = 'data/im.db.zip'

#Extract the SQLite database file from the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('unzippedData')

#connect to extracted SQLite database
database_path = 'unzippedData/im.db'
conn = sqlite3.connect(database_path)

In [236]:
schema_df = pd.read_sql("""

SELECT *
FROM sqlite_master

""", conn)

schema_df

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [249]:
movie_basic = pd.read_sql('''
    SELECT *
    FROM movie_basics
''', conn)
movie_basic

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [238]:
movie_basic["start_year"].value_counts()

2017    17504
2016    17272
2018    16849
2015    16243
2014    15589
2013    14709
2012    13787
2011    12900
2010    11849
2019     8379
2020      937
2021       83
2022       32
2023        5
2024        2
2027        1
2026        1
2025        1
2115        1
Name: start_year, dtype: int64

In [239]:
result_sql = pd.merge(movie_budget_copy[['release_date', 'movie', 'production_budget', 'domestic_gross', 'worldwide_gross', "release_year", "release_month"]], movie_basic[['primary_title', 'original_title', 'start_year', 'genres']], left_on='movie', right_on='primary_title', how='inner')

result_sql

,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month,primary_title,original_title,start_year,genres
0,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,776,345,279",2009,12,Avatar,Abatâ,2011,Horror
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011,5,Pirates of the Caribbean: On Stranger Tides,Pirates of the Caribbean: On Stranger Tides,2011,"Action,Adventure,Fantasy"
2,2019-06-07,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",2019,6,Dark Phoenix,Dark Phoenix,2019,"Action,Adventure,Sci-Fi"
3,2015-05-01,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",2015,5,Avengers: Age of Ultron,Avengers: Age of Ultron,2015,"Action,Adventure,Sci-Fi"
4,2018-04-27,Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,134,200",2018,4,Avengers: Infinity War,Avengers: Infinity War,2018,"Action,Adventure,Sci-Fi"
...,...,...,...,...,...,...,...,...,...,...,...
3810,2001-07-06,Cure,"$10,000","$94,596","$94,596",2001,7,Cure,Cure,2014,None
3811,1996-04-01,Bang,"$10,000",$527,$527,1996,4,Bang,Bang,2015,None
3812,2012-01-13,Newlyweds,"$9,000","$4,584","$4,584",2012,1,Newlyweds,Newlyweds,2011,"Comedy,Drama"
3813,2018-12-31,Red 11,"$7,000",$0,$0,2018,12,Red 11,Red 11,2019,"Horror,Sci-Fi,Thriller"


In [240]:
result_sql['worldwide_gross'] = result_sql['worldwide_gross'].str.replace('$', '').str.replace(',', '').astype(float)
result_sql['production_budget'] = result_sql['production_budget'].str.replace('$', '').str.replace(',', '').astype(float)

result_sql['ROI'] = (result_sql['worldwide_gross'] / result_sql['production_budget']) * 100

result_sql

,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month,primary_title,original_title,start_year,genres,ROI
0,2009-12-18,Avatar,425000000.0,"$760,507,625",2.776345e+09,2009,12,Avatar,Abatâ,2011,Horror,653.257713
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000.0,"$241,063,875",1.045664e+09,2011,5,Pirates of the Caribbean: On Stranger Tides,Pirates of the Caribbean: On Stranger Tides,2011,"Action,Adventure,Fantasy",254.667286
2,2019-06-07,Dark Phoenix,350000000.0,"$42,762,350",1.497624e+08,2019,6,Dark Phoenix,Dark Phoenix,2019,"Action,Adventure,Sci-Fi",42.789243
3,2015-05-01,Avengers: Age of Ultron,330600000.0,"$459,005,868",1.403014e+09,2015,5,Avengers: Age of Ultron,Avengers: Age of Ultron,2015,"Action,Adventure,Sci-Fi",424.384139
4,2018-04-27,Avengers: Infinity War,300000000.0,"$678,815,482",2.048134e+09,2018,4,Avengers: Infinity War,Avengers: Infinity War,2018,"Action,Adventure,Sci-Fi",682.711400
...,...,...,...,...,...,...,...,...,...,...,...,...
3810,2001-07-06,Cure,10000.0,"$94,596",9.459600e+04,2001,7,Cure,Cure,2014,None,945.960000
3811,1996-04-01,Bang,10000.0,$527,5.270000e+02,1996,4,Bang,Bang,2015,None,5.270000
3812,2012-01-13,Newlyweds,9000.0,"$4,584",4.584000e+03,2012,1,Newlyweds,Newlyweds,2011,"Comedy,Drama",50.933333
3813,2018-12-31,Red 11,7000.0,$0,0.000000e+00,2018,12,Red 11,Red 11,2019,"Horror,Sci-Fi,Thriller",0.000000


In [241]:
year = result_sql[result_sql['start_year'] <= 2020]
year

,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month,primary_title,original_title,start_year,genres,ROI
0,2009-12-18,Avatar,425000000.0,"$760,507,625",2.776345e+09,2009,12,Avatar,Abatâ,2011,Horror,653.257713
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000.0,"$241,063,875",1.045664e+09,2011,5,Pirates of the Caribbean: On Stranger Tides,Pirates of the Caribbean: On Stranger Tides,2011,"Action,Adventure,Fantasy",254.667286
2,2019-06-07,Dark Phoenix,350000000.0,"$42,762,350",1.497624e+08,2019,6,Dark Phoenix,Dark Phoenix,2019,"Action,Adventure,Sci-Fi",42.789243
3,2015-05-01,Avengers: Age of Ultron,330600000.0,"$459,005,868",1.403014e+09,2015,5,Avengers: Age of Ultron,Avengers: Age of Ultron,2015,"Action,Adventure,Sci-Fi",424.384139
4,2018-04-27,Avengers: Infinity War,300000000.0,"$678,815,482",2.048134e+09,2018,4,Avengers: Infinity War,Avengers: Infinity War,2018,"Action,Adventure,Sci-Fi",682.711400
...,...,...,...,...,...,...,...,...,...,...,...,...
3810,2001-07-06,Cure,10000.0,"$94,596",9.459600e+04,2001,7,Cure,Cure,2014,None,945.960000
3811,1996-04-01,Bang,10000.0,$527,5.270000e+02,1996,4,Bang,Bang,2015,None,5.270000
3812,2012-01-13,Newlyweds,9000.0,"$4,584",4.584000e+03,2012,1,Newlyweds,Newlyweds,2011,"Comedy,Drama",50.933333
3813,2018-12-31,Red 11,7000.0,$0,0.000000e+00,2018,12,Red 11,Red 11,2019,"Horror,Sci-Fi,Thriller",0.000000


In [242]:
budget_filter = result_sql[result_sql["production_budget"] <= 80000000.0]
budget_filter

,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month,primary_title,original_title,start_year,genres,ROI
20,1998-08-14,The Avengers,60000000.0,"$23,385,416",48585416.0,1998,8,The Avengers,The Avengers,2012,"Action,Adventure,Sci-Fi",80.975693
39,1976-12-17,King Kong,23000000.0,"$52,614,445",90614445.0,1976,12,King Kong,King Kong,2018,"Action,Adventure",393.975848
40,1933-04-07,King Kong,672000.0,"$10,000,000",10000650.0,1933,4,King Kong,King Kong,2018,"Action,Adventure",1488.191964
52,1951-07-28,Alice in Wonderland,3000000.0,$0,0.0,1951,7,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",0.000000
53,1951-07-28,Alice in Wonderland,3000000.0,$0,0.0,1951,7,Alice in Wonderland,Alice in Wonderland,2011,Documentary,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
3810,2001-07-06,Cure,10000.0,"$94,596",94596.0,2001,7,Cure,Cure,2014,None,945.960000
3811,1996-04-01,Bang,10000.0,$527,527.0,1996,4,Bang,Bang,2015,None,5.270000
3812,2012-01-13,Newlyweds,9000.0,"$4,584",4584.0,2012,1,Newlyweds,Newlyweds,2011,"Comedy,Drama",50.933333
3813,2018-12-31,Red 11,7000.0,$0,0.0,2018,12,Red 11,Red 11,2019,"Horror,Sci-Fi,Thriller",0.000000


In [250]:
profit = budget_filter[budget_filter["ROI"] >= 200.0]
profit

,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month,primary_title,original_title,start_year,genres,ROI
39,1976-12-17,King Kong,23000000.0,"$52,614,445",90614445.0,1976,12,King Kong,King Kong,2018,"Action,Adventure",393.975848
40,1933-04-07,King Kong,672000.0,"$10,000,000",10000650.0,1933,4,King Kong,King Kong,2018,"Action,Adventure",1488.191964
65,1999-05-07,The Mummy,80000000.0,"$155,385,488",416385488.0,1999,5,The Mummy,The Mummy,2017,"Action,Adventure,Fantasy",520.481860
71,1974-03-29,The Great Gatsby,7000000.0,"$26,533,200",26533200.0,1974,3,The Great Gatsby,The Great Gatsby,2013,"Drama,Romance",379.045714
82,1992-11-11,Aladdin,28000000.0,"$217,350,219",504050219.0,1992,11,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",1800.179354
...,...,...,...,...,...,...,...,...,...,...,...,...
3797,2015-09-01,Exeter,25000.0,$0,489792.0,2015,9,Exeter,Exeter,2014,Horror,1959.168000
3798,2015-09-01,Exeter,25000.0,$0,489792.0,2015,9,Exeter,Exeter,2015,None,1959.168000
3808,2006-04-28,Clean,10000.0,"$138,711",138711.0,2006,4,Clean,Clean,2017,"Comedy,Drama,Horror",1387.110000
3809,2001-07-06,Cure,10000.0,"$94,596",94596.0,2001,7,Cure,Cure,2011,Drama,945.960000


In [252]:
profit.drop_duplicates(subset="movie", keep="first", inplace=True)
profit

,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month,primary_title,original_title,start_year,genres,ROI
39,1976-12-17,King Kong,23000000.0,"$52,614,445",90614445.0,1976,12,King Kong,King Kong,2018,"Action,Adventure",393.975848
65,1999-05-07,The Mummy,80000000.0,"$155,385,488",416385488.0,1999,5,The Mummy,The Mummy,2017,"Action,Adventure,Fantasy",520.481860
71,1974-03-29,The Great Gatsby,7000000.0,"$26,533,200",26533200.0,1974,3,The Great Gatsby,The Great Gatsby,2013,"Drama,Romance",379.045714
82,1992-11-11,Aladdin,28000000.0,"$217,350,219",504050219.0,1992,11,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",1800.179354
138,1991-11-13,Beauty and the Beast,20000000.0,"$376,057,266",608431132.0,1991,11,Beauty and the Beast,La belle et la bête,2014,"Drama,Fantasy,Romance",3042.155660
...,...,...,...,...,...,...,...,...,...,...,...,...
3790,2016-03-18,Krisha,30000.0,"$144,822",144822.0,2016,3,Krisha,Krisha,2015,Drama,482.740000
3795,2000-01-14,The Terrorist,25000.0,"$195,043",195043.0,2000,1,The Terrorist,The Terrorist,2018,Thriller,780.172000
3796,2015-09-01,Exeter,25000.0,$0,489792.0,2015,9,Exeter,Exeter,2015,"Horror,Mystery,Thriller",1959.168000
3808,2006-04-28,Clean,10000.0,"$138,711",138711.0,2006,4,Clean,Clean,2017,"Comedy,Drama,Horror",1387.110000


In [247]:
profit["movie"].value_counts()

The Grey                            1
Free Birds                          1
Seven Psychopaths                   1
Hell or High Water                  1
Me Before You                       1
                                   ..
Resident Evil: The Final Chapter    1
La La Land                          1
The Shallows                        1
Safety Not Guaranteed               1
Bridesmaids                         1
Name: movie, Length: 857, dtype: int64